# LAB05 - PROJECT4 REPORT: MÔ HÌNH HỒI QUY TUYẾN TÍNH

* **Họ và tên**: Phạm Ngọc Thùy Trang - MSSV: 18127022
* **Lớp**: 18CLC1 - Môn: Toán Ứng dụng và thống kê
* **Giáo Viên Hướng Dẫn:** Trần Thị Thảo Nhi
* **Khoa**: Công Nghệ Thông Tin - HCMUS

1. Import các thư viện cần thiết (đã có sẵn trong file đề và không được phép thay đổi)

In [1]:
# DON'T CHANGE this part: import libraries
import numpy as np
import scipy
import json
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import itertools

2. Đọc đường dẫn truyền vào tập dữ liệu gồm: tập train, tập valid và một con số random bất kì

In [2]:
# DON'T CHANGE this part: read data path
train_set_path, valid_set_path, random_number = input().split()

train.json valid.json 100


## I. XỬ LÝ DỮ LIỆU DẠNG VĂN BẢN

*Bài toán này em làm theo các bước trong TODO list dưới đây để đảm bảo ra được kết quả hoàn chỉnh nhất*

In [3]:
# TODO:
# 1. preprocess: converting text to lowercase, coverting number, tokenization, removing stopword, stemming
# 2. embedding: hitogram matrix
# 3. classifier using linear regression
# 4. accuracy (for metric)
# Em làm theo 4 bước ở đây

* Đọc file dữ liệu gồm file valid và file train
* Download các package "punkt" và "stopwords" trong thư viện nltk có sẵn để
tiền xử lý dữ liệu dạng văn bản

In [4]:
#print(train_set_path)
#print(valid_set_path)
#print(random_number)


In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

* **Hàm read_json(data_path)**: được dùng để đọc file json vì hai tập dữ liệu của chúng ta đều có dạng .json
    - **Tham số đầu vào:** đường dẫn của tập dữ liệu
    - **Tham số trả về:** danh sách các trường "reviewText" và các trường "overall" của từng object trong file

* Sau đó lần lượt gán tập X_train, Y_train là hai giá trị được trả về từ hàm read_json (đối với tập train)

In [6]:
# đọc file json
def read_json(data_path):
    overall_list = []
    review_list = []
    f = open(data_path)
    data = json.load(f) 
    for item in data: 
        overall_list.append(item['overall'])
        review_list.append(item['reviewText'])
    return review_list,overall_list


X_train,y_train = read_json(train_set_path)

#print(len(X_train),len(y_train))
#print(X_train[0])
#print(y_train[0])

#Dữ liệu sau khi chạy hai hàm print ở trên cho tập train X và tập train Y
# 5000 5000
# I needed SHORT cables to connect adjacent units in a rack.  This was a perfect fit.  The colors make signal tracing a matter of sight rather than  scribble strips.
# 5.0

* Gán tập X_valid, Y_valid tương ứng với kết quả trả về từ hàm read_file đối với tập dữ liệu valid

In [7]:
X_valid,y_valid = read_json(valid_set_path)

#print(len(X_valid),len(y_valid))
#print(X_valid[0])
#print(y_valid[0])

#--lấy ra giá trị X và giá trị Y thông qua hàm đọc file json
# print(len(X_valid),len(y_valid)) -- 500 500
# print(X_valid[0]) -- These pens never get old. I prefer medium point. LOVE the mulitple colors.
# print(y_valid[0]) -- 5.0

#### Ví dụ cho phần báo cáo, nên báo cáo cho từng phần code để rõ ràng

Báo cáo phần tiền xử lý: dùng xyz để tách từ, ...

... Đối với những từ out-of-vocab (xuất hiện trong tập train nhưng không có ở tập valid), xử lý bằng cách ... 

### Qúa trình preprocess (tiền xử lý dữ liệu dạng văn bản)

* **Hàm preprocess_train(text)**: dùng để tiền xử lý dạng văn bản, chuyển tất cả các chữ thành chữ thường, chuyển số thành ký tự "num" cho đơn giản, tách từ, loại bỏ stopwords *(các short word như the, is, at; các từ thường xuất hiện như what, this, how, các từ cực kỳ hiếm khi xuất hiện)*, stemming *(biến đổi một từ về dạng gốc, loại bỏ "endings" ra khỏi từ)*, chuyển các từ chưa xuất hiện trong quá trình học thành ký tự "unk"
  1. **Tham số đầu vào:** văn bản chưa được xử lý (dạng thô)
  2. **Tham số đầu ra:** văn bản đã được tiền xử lý bằng cách sử dụng các bước ở trên như mô tả

* **Hàm preprocess(X_data)**: truyền vào trong nó là một tập dữ liệu, hàm này được dùng để tiền xử lý cho tập train *(có thể xem thêm ở phần print(pre_text[int(random_number)])*. Thay vì trả về một danh sách gồm các đoạn văn trong tập train như ở trên, hàm này chỉ trả về một câu hay một đoạn văn bản đã được tiền xử lý dựa trên con số random mà ta nhập vào trước đó. Dan**h sách trả về là một list**

In [8]:
# Ví dụ cho phần code hàm tiền xử lý tập train
# 1. preprocess: converting text to lowercase, coverting number, tokenization, removing stopword, stemming
# Đầu vào là 1 văn bản, đầu ra văn bản đã được tiền xử lý sử dụng các bước ở trên.
stemmer= PorterStemmer()
stop_words = set(stopwords.words("english"))
def preprocess_train(text):
    # converting text to lowercase
    text = text.lower()
    # Chuyển giá trị số về num
    text = re.sub(r"[0-9]+", " num ", text)
    # Tokenizer
    tokens = word_tokenize(text)
    # xoá stopword
    text = [i for i in tokens if not i in stop_words]
    # Stemming
    text = [stemmer.stem(word) for word in text]
    return text

def preprocess(X_data):
    list_new = []
    for x in X_data:
        x_processed = preprocess_train(x)
        list_new.append(x_processed)
    return list_new

pre_text = preprocess(X_train)
#print(pre_text[int(random_number)])


* Đoạn code này được dùng để lấy toàn bộ vocabulary trong tập train để kiểm tra trong tập test và thay bằng giá trị unk

* Có thể chạy thử hai câu lệnh print(len(vocab)) và print(vocab) để biết được số lượng từ vựng trong tập train cũng như các từ trong tập train


In [9]:
# Lấy toàn bộ vocabulary trong tập train để kiếm tra trong tập test và thay bằng giá trị unk
vocab = ["unk"]
for item in pre_text:
    for token in item:
        if token not in vocab:
            vocab.append(token)
#print("Số lượng từ vựng trong tập train: ", len(vocab))
#print(vocab): các từ trong tập train

In [10]:
#print(X_valid) --['These pens never get old. I prefer medium point. LOVE the mulitple colors.', "Not as strong as I would have liked as is my Peavey mike but they'll do for now for the price.", 'Great calendar but the bottom edge of the sheets will wrinkle and fray with use. If there was a protective bottom edge it would make this a great product.',...]

* **Hàm preprocess_testData(text,vocab)**: hàm này được dùng để xử lý trên tập valid *(tập test)*. Bởi vì đề có yêu cầu các từ nào không xuất hiện trong tập train thì thay bằng giá trị "unk". Do đó chúng ta chúng ta cần xây dựng 1 hàm riêng với đầu vào là 1 văn bản và tập từ điển trong tập train để kiểm tra xem cái từ trong tập test có tồn tại trong từ điển tập train không. Nếu tồn tại thì giữ nguyên. Không tồn tại thì thay bằng giá trị "unk". Qúa trình xử lý tiền văn bản tương tự như hàm preprocess_train.
  - **Tham số đầu vào**: văn bản chưa được xử lý (dạng thô) và tập vocab
  - **Tham số trả về:** văn bản đã được tiền xử lý bằng cách sử dụng các bước ở trên như mô tả

* **Hàm preprocess(X_valid, vocab):** truyền vào trong nó là một tập dữ liệu và tập vocab, hàm này được dùng để tiền xử lý cho tập valid. Nếu như hàm trên được dùng để xử lý trên từng văn bản thì hàm này là duyệt toàn bộ dữ liệu trong tập valid/tập test. Danh sách trả về là một list.
  - **Tham số đầu vào:** tập valid, tập vocab
  - **Tham số trả về:** một list đã được xử lý. Từ đây ta có thể in ra câu thứ random_number trong tập valid

In [11]:
# 1. preprocess: converting text to lowercase, coverting number, tokenization, removing stopword, stemming
def preprocess_testData(text,vocab):
    # converting text to lowercase
    text = text.lower()
    # Chuyển giá trị số về num
    text = re.sub(r"[0-9]+", " num ", text)
    # Tokenizer
    tokens = word_tokenize(text)
    # Remove stopword
    text = [i for i in tokens if not i in stop_words]
    # Stemming
    text = [stemmer.stem(word) for word in text]
    # chuyển chữ trong tập test thành giá trị unk
    list_new = []
    for token in text:
        if token in vocab:
            list_new.append(token)
        else:
            list_new.append("unk")
    text = list_new
    return text

# Hàm trên là xử lý ttừng văn bản. Còn hàm này là duyệt toàn bộ dữ liệu trong tập valid hoặc tập test.
# Danh sách là trả về một list.
def preprocess_test(X_valid,vocab): 
    list_process = []
    for item in X_valid:
        x_processed = preprocess_testData(item,vocab)
        list_process.append(x_processed)
    return list_process

pre_text_valid = preprocess_test(X_valid,vocab)
print(pre_text_valid[int(random_number)]) #in câu thứ random_number của tập valid (từ valid_set_path)

['legal', 'pad', 'throughout', 'offic', 'differ', 'note', '.', 'regular', 'legal', 'pad', 'get', 'job', 'done', 'top', 'gold', 'unk', 'pad', 'great', 'heavi', 'feel', 'sheet', 'nice', 'stock', '.']


### Xây dựng bộ ma trận nhúng của văn bản (document embedding): mỗi văn bản là 1 vector histogram (có chiều bằng chiều dài bộ từ điển vocab_size)

Có nhiều cách để chúng ta xây dựng vector hóa văn bản, nhưng ở đây ta buộc phải dùng histogram vector

* **Hàm histogram_vector(document, vocab)**: Với mỗi văn bản là một vector histogram có chiều dài bằng chiều dài bộ từ điển vocab và với ý nghĩa là mỗi thành phần i chứa tần suất xuất hiện của từ thứ i trong document đang xét *(có thể dùng câu lệnh histogram_vector(s,["i","love","baby"]) để kiểm tra)*. Hàm này được xây dựng dựa trên công thức histogram vector (Chương Document Count Vector - Slide tham khảo của cô - trang 20)
    - **Tham số đầu vào**: document bất kỳ, tập từ vựng vocab
    - **Tham số trả về**: một vector histogram với chiều dài bằng chiều dài bộ từ điển vocab có chứa mỗi phần tử i là tần suất xuất hiện của từ thứ i trong document đang xét

In [12]:
# câu 2 . Histogram. Đầu vào là 1 văn bản, đầu ra là vector histogram. Document embedding đó.
# Ta xem lại cái công thức histogram vector trong slide
def histogram_vector(document,vocab):
    vector = np.zeros(len(vocab))
    vector_one = np.ones(len(vocab))
    for index,word in enumerate(vocab):
        if word in document:
            vector[index] = document.count(word)
    histogram = vector/(vector_one.T @ vector)
    return histogram
# Ví dụ
s = "i love you baby baby".split(' ')
#histogram_vector(s,["i","love","baby"])

In [13]:
#print(pre_text[0])

* **Hàm matrix_histogram(X_data, vocab)**: tính giá trị histogram cho toàn bộ dữ liệu và xây dựng bộ ma trận nhúng của văn bản. Có thể dùng câu lệnh print(histogram_X_train[0]) để hiển thị ví dụ document embedding hay câu lệnh print(len(histogram_X_train[0])) để hiển thị độ dài của một vector histogram để kiểm chứng
    - **Tham số đầu vào:** danh sách các dữ liệu từng tập, bộ từ vựng vocab
    - **Tham số đầu ra:** ma trận histogram document cho toàn bộ dữ liệu

In [14]:
# Tính giá trị histogram cho toàn bộ tập dữ liệu. Đầu vào là danh sách dữ liệu từng tập. Đầu ra là matrận histogram document cho toàn bộ dữ liệu
def matrix_histogram(X_data,vocab):
    matrix = []
    for item in X_data:
        matrix.append(histogram_vector(item,vocab))
    return matrix

histogram_X_train = matrix_histogram(pre_text,vocab)
histogram_X_valid = matrix_histogram(pre_text_valid,vocab)

In [15]:
# Hiện thị ví dụ document embedding (histogram vector)
# print(histogram_X_train[0]) --[0.         0.04761905 0.04761905 ... 0.         0.         0.        ]
# Hiện thị độ dài của 1 vector hisgram. Nó là chiều của tập từ điển vocab. Xem video trình bày Cô sẽ hiểu cái.
# print(len(histogram_X_train[0])) --6523

### Chuẩn hóa nhãn thành dạng số hoặc vector

* **Hàm category_vector(y)**: Được dùng để chuyển nhãn thành vector, đầu vào là một nhãn bất kỳ và đầu ra là một vector tương ứng

* **Hàm convert_y(y_data):** Hàm này được dùng để chuyển toàn bộ nhãn y trong tập train và trong tập valid thành vector 5 chiều, và kết quả trả về tất nhiên là một ma trận. Ví dụ nhãn 1.0 sẽ có kết quả là [1,0,0,0,0], nhãn 5.0 sẽ có kết quả là [0,0,0,0,1]

* *Ta có thể chạy thử dòng lệnh print(y_train[0]) và print(y_train_encode[0]) để kiểm tra kết quả*

In [16]:
# chuyển nhãn thành vector
def category_vector(y):
    vector = np.zeros(5)
    vector[int(y)-1] = 1.0
    return vector

# Hàm chuyển toàn bộ nhãn y trong tập train và tập valid thành vector 5 chiều.
def convert_y(y_data):
    matrix = []
    for item in y_data:
        matrix.append(category_vector(item))
    return matrix
y_train_encode = convert_y(y_train)
y_valid_encode = convert_y(y_valid)
# Hiện thị ví dụ ở đây.
# print(y_train[0]) --5.0
# print(y_train_encode[0]) --[0. 0. 0. 0. 1.]

## II. SỬ DỤNG MÔ HÌNH HỒI QUY TUYẾN TÍNH DÙNG BÌNH PHƯƠNG TỐI TIỂU - XÂY DỰNG MÔ HÌNH M2

* Bằng cách dùng mô hình hồi quy tuyến tính dùng bình phương tối tiểu, ta có thể xây dựng một mô hình phức tạp M2 dự đoán nhãn theo trường overall từ 1 - 5. Với mô hình này, kết quả của hồi quy tuyến tính một vector Y 5 chiều, index của chiều đại diện cho từng nhãn, thông qua hàm softmax để trả về kết quả phù hợp

* Kết quả của hồi quy tuyến tính một vector Y 5 chiều, index của chiều đại diện cho từng nhãn đã được thực hiện ở phần trên

* **Hàm linear_regression(X_train,y_train):** Với đầu vào là tập dữ liệu X_train, Y_train và đầu ra là giá trị theta. Hàm này xây dựng dựa trên mô hình hồi quy tuyến tính dùng bình phương tối tiểu

* Ta có thể chạy thử lệnh print("theta : ", theta) để kiểm tra giá trị theta

In [17]:
# Hàm linear regression như ở lab 4. Đầu vào là x_train, y_train. đầu ra là giá trị theta
def linear_regression(X_train,y_train):
    theta = np.linalg.pinv(X_train) @ y_train
    return theta
# Huấn luyện trên dữ liệu X_train
theta = linear_regression(histogram_X_train,y_train_encode)
#print("theta : ", theta)
#print(len(theta)) --6523

# theta :  [[ 9.86483024e-12  5.51049173e-11 -2.29972476e-11 -3.01103150e-11
#   -1.15498186e-11]
#  [ 4.17777377e-02 -6.15464223e-02 -1.64461538e-01  2.43738281e-02
#    1.15985639e+00]
#  [ 1.68808765e+00  1.43154843e-01 -1.00180007e-01  3.50076476e+00
#   -4.23182724e+00]
#  ...
#  [ 7.52603474e+00  9.98995941e+00 -9.06672293e+00  3.36612294e+01
#   -4.08743587e+01]
#  [-6.74111785e+00  2.03874042e+01  2.57644048e+01 -3.58669515e+01
#   -2.94373973e+00]
#  [-1.65997390e+00  3.36241697e-01  2.66890431e+00  1.05087079e+01
#   -1.08538800e+01]]

* **Hàm softmax(x):** Từ vector Y 5 chiều với index của chiều đại diện cho từng nhãn, thông qua hàm này để trả ra nhãn phù hợp. Công thức của softmax và chi tiết của nó ta có thể xem thêm [tại đây](https://machinelearningcoban.com/2017/02/17/softmax/) . Với đầu vào là một ma trận với mỗi cột là một vector x đầu ra cũng là một ma trận mà mỗi cột có giá trị là softmax(x). Các giá trị của x còn được gọi là scores

In [18]:
# Hàm softmax vì ở đây ta làm câu 5 nhãn (Mô hình M2. 10đ)
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)


* **Hàm predict(theta, input):** Hàm này được dùng để dự đoán trên tập test
  - **Tham số đầu vào:** theta, histogramX thứ i của tập valid 
  - **Tham số trả về:** nhãn dự đoán

* Đoạn code dưới nữa còn được dùng để chạy dự đoán nhãn trên tập valid (tập test)
và lưu vào danh sách. Ta có thể kiểm chứng bằng cách cho chạy dòng lệnh print(y_pred)*(nhãn dự đoán của mô hình linear)* và print(y_valid) *(nhãn đúng của nó)* để so sánh



In [19]:
# Sử dụng hàm softmax trước khi xác định nhãn.
def predict(theta,input):
    predict_value = np.dot(input, theta)
    output = softmax(predict_value)
    label = np.where(output == np.amax(output))[0][0] + 1
    return label

# Chạy dự đoán trên tập test và lưu vào danh sách.
y_valid = [int(x) for x in y_valid]
y_pred = []
for index in range(len(histogram_X_valid)):
    y_pred.append(predict(theta,histogram_X_valid[index]))
# hàng ở trên là nhãn dự đoán của mô hình linear
# Hàng ở dưới là nhãn đúng của nó.

#print(y_pred) --[4, 5, 4, 4, 5, 5, 5, 1, 5, 4, 5, 4, 5, 5, 4, 5, 5, 4, 3, 5, 5, 5, 2, 3, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 3, 5, 2, 5, 5, 3, 5, 5, 4, 5, 4, 4, 4, 5, 5, 5, 5, 4, 5, 5, 4, 5, 3, 5, 5, 5, 5, 3, 3, 5, 4, 5, 3, 5, 5, 4, 4, 5, 5, 3, 3, 1, 5, 1, 3, 5, 4, 5, 3, 4, 4, 5, 4, 5, 4, 5, 4, 5, 3, 4, 4, 5, 5, 4, 4, 5, 2, 4, 5, 3, 5, 5, 5, 1, 5, 5, 2, 5, 4, 5, 5, 5, 4, 2, 5, 5, 4, 5, 5, 5, 5, 4, 4, 5, 2, 5, 5, 5, 4, 4, 5, 4, 5, 5, 1, 1, 1, 4, 5, 5, 5, 3, 1, 5, 3, 2, 3, 5, 5, 4, 5, 5, 4, 4, 5, 5, 5, 4, 5, 2, 5, 2, 5, 5, 5, 5, 5, 5, 4, 5, 4, 2, 5, 3, 1, 3, 5, 3, 3, 5, 4, 4, 5, 5, 1, 5, 5, 3, 5, 5, 5, 3, 3, 4, 4, 5, 5, 5, 3, 4, 2, 5, 4, 4, 3, 5, 1, 5, 4, 5, 4, 5, 2, 5, 4, 5, 5, 5, 5, 5, 4, 5, 5, 4, 4, 5, 5, 4, 5, 5, 4, 5, 5, 5, 4, 5, 1, 4, 5, 5, 3, 5, 5, 4, 5, 4, 5, 5, 5, 4, 5, 4, 3, 5, 5, 5, 4, 5, 5, 5, 4, 5, 4, 4, 5, 5, 3, 4, 5, 5, 3, 5, 4, 5, 5, 2, 5, 5, 5, 4, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4, 5, 3, 5, 1, 3, 5, 5, 3, 4, 2, 3, 5, 5, 5, 4, 5, 3, 5, 4, 5, 3, 5, 5, 2, 5, 4, 5, 5, 5, 5, 2, 5, 2, 4, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 2, 5, 5, 1, 5, 5, 5, 5, 5, 4, 4, 3, 2, 5, 5, 5, 4, 4, 5, 5, 5, 3, 3, 3, 4, 2, 4, 5, 2, 2, 5, 5, 5, 4, 5, 5, 5, 5, 3, 4, 2, 1, 3, 5, 5, 5, 3, 5, 4, 5, 5, 5, 5, 5, 5, 4, 5, 4, 3, 4, 4, 5, 5, 4, 5, 5, 5, 5, 3, 3, 4, 5, 5, 3, 5, 5, 5, 5, 3, 4, 5, 5, 5, 4, 4, 5, 1, 5, 5, 5, 5, 5, 5, 3, 5, 2, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, 4, 5, 4, 5, 5, 4, 1, 1, 5, 5, 5, 5, 5, 3, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 3, 4, 5, 1, 3, 3, 3, 5, 1, 5, 3, 5, 5, 5, 5, 4, 4, 5, 5]
#print(y_valid) -- [5, 3, 3, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 1, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 4, 4, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 1, 5, 4, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 3, 5, 3, 5, 5, 4, 5, 1, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 4, 5, 5, 3, 5, 5, 5, 5, 4, 5, 5, 5, 5, 2, 4, 5, 1, 1, 5, 5, 5, 5, 2, 5, 4, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 3, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 2, 5, 2, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5, 4, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 4, 5, 4, 5, 5, 1, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 3, 2, 5, 5, 1, 1, 5, 5, 5, 5, 5, 4, 2, 5, 4, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 1, 4, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 5, 5, 4, 4, 3, 3, 3, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1, 2, 5, 5, 3, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 4, 5, 5, 5, 5, 5, 1, 4, 3, 5, 5, 5, 5, 5, 5, 3, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 4, 5, 4, 1, 4, 4, 5, 5, 5, 5, 3, 1, 5, 5, 1, 5, 4, 4, 3, 4, 3, 5, 5, 5, 3, 5, 3, 5, 5, 5, 5, 5, 5]

## III. SỬ DỤNG ĐỘ CHÍNH XÁC ĐỂ ĐÁNH GIÁ MÔ HÌNH

### Mô hình được đánh giá bằng độ chính xác (accuracy)

* **Hàm accuracy_score(y_pred,y_test):** Hàm này được dùng để tính độ chính xác của mô hình phân lớp dựa trên số nhãn dự đoán được, giá trị trả về sẽ * 100%

In [20]:
# Tính độ chính xác * 100%.
def accuracy_score(y_pred, y_test):
    count = 0
    for index,value in enumerate(y_test):
        if value == y_pred[index]:
           count +=1
    acc = count/len(y_test)
    return acc
accuracy = accuracy_score(y_pred,y_valid)
print("M2 - ", accuracy * 100)

M2 -  51.0


## IV. TÀI LIỆU THAM KHẢO

1. https://machinelearningcoban.com/2017/02/17/softmax/
2. Slide PDF hướng dẫn cho đồ án của cô Trần Thị Thảo Nhi
3. https://www.geeksforgeeks.org/removing-stop-words-nltk-python/